### Visualize strokes
This notebook is designed for exploratory analysis by visualizing user strokes.

In [1]:
EXPERIMENT_GROUP = "0.1" # Set this to determine which experiment to visualize

ROOT_DIR = "../../.."
DATA_DIRECTORY = "data_experiment/laps_{}/raw"
DATA_FILE = "raw_experiment_data.json"

IMAGES_BASE_DIRECTORY = "static/images"

USER_STROKES_DIRECTORY = "data_experiment/laps_{}/strokes"

import os, json, argparse, random, copy
import pathlib
from datetime import datetime
from collections import defaultdict

image_directory = os.path.join(ROOT_DIR, IMAGES_BASE_DIRECTORY)
experiment_file = os.path.join(ROOT_DIR, DATA_DIRECTORY.format(EXPERIMENT_GROUP), DATA_FILE)
print(f"Visualizing data from experiment file: {experiment_file}")
with open(experiment_file) as f:
    experiment_data = json.load(f)

Visualizing data from experiment file: ../../../data_experiment/laps_0.1/raw/raw_experiment_data.json


In [2]:
experiment_ids = experiment_data['metadata']['experiment_ids']
experiments = experiment_data['experiment_ids']
ALL = "all"

%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

UsageError: Line magic function `%%javascript` not found.


#### Experiment summary statistics

In [3]:
for experiment_id in experiment_ids:
    print(experiment_id)
    for condition in experiments[experiment_id]['conditions']:
        print(f"\t{condition}: {len(experiments[experiment_id]['conditions'][condition])} subject")

0_baselines_priors__train-none__test-default__neurips_2020
	all: 1 subject
1_no_provided_language__train-im-dr__test-default__neurips_2020
3_producing_language__train-im-de__test-default__neurips_2020
3_producing_language__train-im-dr-de__test-default__neurips_2020


#### Visualize images and strokes

In [4]:
# Drawgood utility functions
import sys
sys.path.append('../..')

from drawgoodlib import utils

In [9]:
def get_user_stroke_images(experiment_id, condition, user_id, user_images, user_strokes):
    user_stroke_images = []
    for img_idx, img in enumerate(user_images):
        if img is None: continue
        stroke_img_file = f"{user_id}_{os.path.basename(img)}"
        stroke_img_dir = os.path.join(ROOT_DIR, USER_STROKES_DIRECTORY.format(EXPERIMENT_GROUP), experiment_id, condition)
        full_stroke_img_path = os.path.join(stroke_img_dir, stroke_img_file)
        user_stroke_images.append(full_stroke_img_path)
        if not os.path.exists(full_stroke_img_path):
            pathlib.Path(stroke_img_dir).mkdir(parents=True, exist_ok=True)
            raw_strokes = user_strokes[img_idx]
            stroke_data = utils.process_stroke_data(raw_strokes)
            utils.saveDrawing(stroke_data, full_stroke_img_path)
    return user_stroke_images
            

In [40]:
from IPython.display import HTML, Image

def _src_from_data(data):
    """Base64 encodes image bytes for inclusion in an HTML img element"""
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

def visualizer_gallery_html(images, stroke_images, descriptions, row_height='auto'):
    """Shows a set of images in a gallery that flexes with the width of the notebook.
    """
    figures = []
    for image_idx, image in enumerate(images):
        if isinstance(image, bytes):
            original_image = _src_from_data(image)
            caption = ''
        else:
            original_image = image
            img_description = descriptions[image_idx]
            stroke_image = stroke_images[image_idx]
            caption = f'<div style="font-size: 1em; width: {row_height};">{img_description}</div>'
        figures.append(f'''
            <div style="display: block">
            <img src="{stroke_image}" style="height: {row_height};">
            {caption}
            </div>
           
        ''')
    return f'''
        <div style="display: flex; flex-flow: row wrap;">
        {''.join(figures)}
        </div>
    '''

def text_html(text):
    return f"<div>{text}</div>"

html = ""
experiments_to_load = [ALL]
for experiment_id in experiment_ids:
    should_load = experiment_ids in experiments_to_load or ALL in experiments_to_load
    this_experiment = experiments[experiment_id]
    has_users = this_experiment['summary']['total_users'] > 0
    if not should_load: continue
    if not has_users: continue
    html += text_html(f"Visualizing strokes for: {experiment_id}")
    
    for condition in this_experiment['conditions']:
        html += text_html(f"Condition: {condition}")
        condition_users = this_experiment['conditions'][condition]
        for idx, user_id in enumerate(condition_users):
            html += text_html(f"\nUser {idx}/{len(condition_users)}")
            user_images = this_experiment['images'][user_id]
            user_descriptions = this_experiment['descriptions'][user_id]
            user_strokes = this_experiment['strokes'][user_id]
            
            user_images = [os.path.join(image_directory, img) for img in user_images if img is not None]
            
            stroke_images = get_user_stroke_images(experiment_id, condition, user_id, user_images, user_strokes)
            
            html += visualizer_gallery_html(user_images, stroke_images, user_descriptions, row_height='200px')

HTML(data=html)
    